## 6. Optimization

학습 loop
  1. optimizer.zero_grad()를 호출해 모델 parameter의 변화도 재설정
  2. loss.backwards()를 호출해 역전파 실시
  3. optimizer.step()을 호출해 매개변수 조정

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

**Pre-requiosoe code**

In [3]:
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

train_loader = DataLoader(training_data, batch_size=64)
test_loader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [10]:
model = NeuralNetwork()

**Hyperparameter**

In [5]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

**Optimization implementation**

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Forward Feed
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss = loss.item()
            current = batch + 1
            print(f'\rloss: {loss:>7f} [{current:>5d}]', end='')

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f'\nTest accuracy: {100 * correct :>0.1f}, avg loss: {test_loss:>8f}\n')

**Training**

In [11]:
loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f'Epoch {t + 1}')
    print('-------------------------------')
    train_loop(train_loader, model, loss_fn, optimizer)
    test_loop(test_loader, model, loss_fn)
print('Done!')

Epoch 1
-------------------------------
loss: 0.500678 [  901]
Test accuracy: 84.2, avg loss: 0.425896

Epoch 2
-------------------------------
loss: 0.420073 [  901]
Test accuracy: 85.5, avg loss: 0.388538

Epoch 3
-------------------------------
loss: 0.359169 [  901]
Test accuracy: 85.5, avg loss: 0.387509

Epoch 4
-------------------------------
loss: 0.345636 [  901]
Test accuracy: 87.0, avg loss: 0.347132

Epoch 5
-------------------------------
loss: 0.340556 [  901]
Test accuracy: 86.5, avg loss: 0.374711

Epoch 6
-------------------------------
loss: 0.318372 [  901]
Test accuracy: 87.7, avg loss: 0.339629

Epoch 7
-------------------------------
loss: 0.323564 [  901]
Test accuracy: 87.3, avg loss: 0.348550

Epoch 8
-------------------------------
loss: 0.259408 [  901]
Test accuracy: 87.8, avg loss: 0.354430

Epoch 9
-------------------------------
loss: 0.299649 [  901]
Test accuracy: 88.4, avg loss: 0.359579

Epoch 10
-------------------------------
loss: 0.257498 [  901]
